In [1]:
import pandas as pd

In [5]:
df=pd.read_csv('Preprocessing1.csv')
df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Month,WeekDay,Day
0,Jet Airways,Delhi,Banglore,DEL → BOM → COK,20:00,04:25 10 Jun,26h 35m,1 stop,In-flight meal not included,14924,6,Thursday,6.0
1,Jet Airways,Delhi,Cochin,DEL → BOM → COK,16:00,19:00 10 Jun,27h,1 stop,In-flight meal not included,10577,6,Sunday,9.0
2,Jet Airways,Mumbai,Hyderabad,BOM → HYD,19:35,21:05,1h 30m,non-stop,No info,5678,3,Friday,15.0
3,Multiple carriers,Delhi,Banglore,DEL → BOM → COK,18:55,01:30 16 Jun,15h 10m,1 stop,In-flight meal not included,7408,5,Monday,6.0
4,Air India,Delhi,Cochin,DEL → COK,17:10,17:55,8h 20m,non-stop,No info,6724,6,Monday,24.0


What is the average of the flight ticket price? Write your answer correct to two decimal places.

In [4]:
df['Price'].mean()

9027.895555555555

During which month did the highest number of flights occur? Months are represented by numerical codes, with January corresponding to 1, February to 2, and so forth.

In [5]:
df['Month'].mode()

0    5
Name: Month, dtype: int64

Is the average price of flight tickets higher on weekends (Saturday and Sunday) or on weekdays (Remaining 5 days)?

In [6]:
week_day = df['WeekDay'].isin(['Monday', 'Tuesday', 'Wednesday','Thursday','Friday'])
week_end = df['WeekDay'].isin(['Saturday','Sunday'])
df[week_day]['Price'].mean()>df[week_end]['Price'].mean()

False


Two of the entries in the 'Additional_Info' column are 'No info' and 'No Info'. Replace all occurrences of 'No Info' with 'No info'. How many flights fall under airline 'IndiGo' and have 'No info' as additional information?

In [9]:
df.loc[df.Additional_Info=='No Info',['Additional_Info']]='No info'
criteria=(df['Airline']=='IndiGo')&(df['Additional_Info']=='No info')
df[criteria]

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Month,WeekDay,Day
8,IndiGo,Mumbai,Hyderabad,BOM → HYD,19:05,20:35,1h 30m,non-stop,No info,2754,5,Monday,6.0
10,IndiGo,Delhi,Cochin,DEL → BOM → COK,09:00,01:30 10 May,10h,1 stop,No info,8854,6,Thursday,27.0
12,IndiGo,Delhi,Cochin,DEL → BOM → COK,17:30,01:30 16 May,8h,1 stop,No info,6979,5,Wednesday,15.0
18,IndiGo,Chennai,Banglore,MAA → CCU,16:40,14:00,7h 35m,non-stop,No info,3858,5,Monday,6.0
35,IndiGo,Banglore,Delhi,BLR → DEL,22:10,01:00 22 May,2h 50m,non-stop,No info,3625,5,Tuesday,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9425,IndiGo,Mumbai,Cochin,BOM → HYD,16:10,10:40,6h 10m,non-stop,No info,4575,3,Sunday,3.0
9427,IndiGo,Chennai,Kolkata,MAA → CCU,22:05,00:25 10 May,2h 20m,non-stop,No info,5277,5,Thursday,9.0
9429,IndiGo,Kolkata,Banglore,CCU → BLR,09:35,12:20,2h 45m,non-stop,No info,4462,3,Monday,18.0
9438,IndiGo,Banglore,Delhi,BLR → DEL,04:00,06:50,2h 50m,non-stop,No info,3943,5,Friday,3.0


Convert the values of 'Duration' into seconds. Enter the average duration (in seconds) of a flight. Enter your answer correct to two decimal places.

In [10]:
# Function to convert 'Duration' to seconds
def duration_to_seconds(duration):
    hours, minutes = 0, 0
    if 'h' in duration:
        hours = int(duration.split('h')[0])
    if 'm' in duration:
        minutes = int(duration.split('m')[0].split()[-1])
    return hours * 3600 + minutes * 60

# Apply the conversion function to the 'Duration' column
df['Duration_seconds'] = df['Duration'].apply(duration_to_seconds)
# Calculate the average duration in seconds
df['Duration_seconds'].mean()

38957.93650793651

Transform the values in the 'dep_time' and 'arrival_time' columns to represent the hour component. For instance, if an entry is 10:05 June 13 or 10:05, the corresponding value should be 10. 
Then convert the time into four categories as follows:
5 <= hour < 12 = Morning 
12 <= hour < 17 = Afternoon 
17 <= hour < 20 = Evening 
20 <= hour < 5 = Night

In [ ]:
def categorize_time(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 20:
        return 'Evening'
    else:
        return 'Night'

# Transform 'Dep_Time' and 'Arrival_Time'
df['Dep_Hour'] = pd.to_datetime(df['Dep_Time']).dt.hour
df['Arrival_Hour'] = pd.to_datetime(df['Arrival_Time'].str.split(' ').str[0]).dt.hour

# Categorize the hours
df['Dep_Time_Category'] = df['Dep_Hour'].apply(categorize_time)
df['Arrival_Time_Category'] = df['Arrival_Hour'].apply(categorize_time)
df.head()

How many flights started in the Morning and arrived the destination at Evening?

In [ ]:
count=(df['Arrival_Time_Category']=='Evening') & (df['Dep_Time_Category']=='Morning')
df[count].describe()

Encode the values of column 'WeekDay' as follows:
Weekends (Sunday, Saturday) = 1
all remaining five days = 0
What is the most frequent (mode) WeekDay?

In [11]:
df['WeekDay_Encoded'] = df['WeekDay'].isin(['Sunday', 'Saturday']).astype(int)
df['WeekDay_Encoded'].mode()

0    0
Name: WeekDay_Encoded, dtype: int32

In [12]:
df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Month,WeekDay,Day,Duration_seconds,WeekDay_Encoded
0,Jet Airways,Delhi,Banglore,DEL → BOM → COK,20:00,04:25 10 Jun,26h 35m,1 stop,In-flight meal not included,14924,6,Thursday,6.0,95700,0
1,Jet Airways,Delhi,Cochin,DEL → BOM → COK,16:00,19:00 10 Jun,27h,1 stop,In-flight meal not included,10577,6,Sunday,9.0,97200,1
2,Jet Airways,Mumbai,Hyderabad,BOM → HYD,19:35,21:05,1h 30m,non-stop,No info,5678,3,Friday,15.0,5400,0
3,Multiple carriers,Delhi,Banglore,DEL → BOM → COK,18:55,01:30 16 Jun,15h 10m,1 stop,In-flight meal not included,7408,5,Monday,6.0,54600,0
4,Air India,Delhi,Cochin,DEL → COK,17:10,17:55,8h 20m,non-stop,No info,6724,6,Monday,24.0,30000,0
